In [1]:
# -*- coding: utf-8 -*-

In [2]:
!pip3 install beautifulsoup4
!pip install mysqlclient
!pip install mysql-connector-python-rf

You should consider upgrading via the 'C:\Users\Marwan Eid\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import datetime
import MySQLdb
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import concurrent.futures
from concurrent.futures import Future 
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
%matplotlib inline

In [4]:
def debugging():
    print("Including nulls too; just making sure sizes match!")
    print("No. of pages->", len(pages_contents))
    print("No. of movies->", len(movies_contents))
    print("No. of occurances of existence of actors in movies->", len(actors_contents))
    print("No. of cast members->", len(cast_contents))
    print("No. of movie names->", len(m_name))
    print("No. of movie durations->", len(m_duration))
    print("No. of movie ratings->", len(m_rating))
    print("No. of movie release_dates->", len(m_release_date))
    print("No. of movie storylines->", len(m_storyline))
    print("No. of movie revenues->", len(m_revenue))
    print("No. of movie genres->", len(m_genres))
    print("No. of movie directors->", len(m_director_ID))
    print("No. of movie writers->", len(m_writer_ID))
    print("No. of cast members", len(cm_ids))
    print("No. of cast first names", len(cm_first_name))
    print("No. of cast last names", len(cm_last_name))
    print("No. of cast biography", len(cm_biography))
    print("No. of cast birthdate", len(cm_birthdate))
    print("No. of cast nationality", len(cm_nationality))

In [5]:
def retrieve_one_page_content(pageNo):
    return requests.get("https://elcinema.com/en/index/work/country/eg?page="+str(pageNo)).content

In [6]:
def retrieve_all_pages_contents():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(retrieve_one_page_content, pageNo): pageNo for pageNo in range(5, 105)}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                pages_contents.append(data)
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                pass

In [7]:
def retrieve_movies_urls_from_contents():
    for i in range(len(pages_contents)):
        soup = BeautifulSoup(pages_contents[i], 'html5lib')
        for movie in soup.findAll(string=re.compile("Movie")):
            for link in movie.find_parents("tr"):
                if (link.a['href'] != "/en/work/2012305/"):
                    movies_urls.append("https://elcinema.com" + link.a['href'])

In [8]:
def list_duplicates(seq):
    tally = defaultdict(list)
    for index, item in enumerate(seq):
        tally[item].append(index)
    return ((key,locs) for key,locs in tally.items() if len(locs) > 1)

In [9]:
def retrieve_one_movie_content(murl):
    return requests.get(murl).content

In [10]:
def retrieve_all_movies_contents():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(retrieve_one_movie_content, murl): murl for murl in movies_urls}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                movies_contents.append(data)
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                pass

In [11]:
def retrieve_movies_attributes_from_contents():
    for i in range(len(movies_contents)):
        soup = BeautifulSoup(movies_contents[i], 'html5lib')
        duration_flag, rating_flag, release_date_flag, revenue_flag, storyline_flag, genres_flag, director_flag, writer_flag = False, False, False, False, False, False, False, False
        link = soup.find_all(href=True)
        m_IDs.append(int(link[1]['href'][-8:-1]))
        for item in soup.find_all(attrs={'class': 'panel jumbo'}):
            name = item.find('span', attrs={'class':'left'})
            if (name != None):
                name = name.text
                m_name.append(name.strip(' \n\t\r'))
            else:
                m_name.append(None)
        for item in soup.find_all(attrs={'class': 'intro-box'}):
            date = None
            for index4, item1 in enumerate(item.find_all(attrs={'class': 'list-separator list-title'})):
                if (index4 == 0):
                    for index5, child5 in enumerate(item1.find_all('li')):
                        if (index5 == 0):
                            if (str(child5.text) != "Director:"):
                                break
                        elif (index5 == 1):
                            dir_id = str(child5.a['href'])[-8:-1]
                            m_director_ID.append(dir_id)
                            director_flag = True
                elif (index4 == 1):
                    for index5, child5 in enumerate(item1.find_all('li')):
                        if (index5 == 0):
                            if (str(child5.text) != "Writer:"):
                                break
                        elif (index5 == 1):
                            w_id = str(child5.a['href'])[-8:-1]
                            m_writer_ID.append(w_id)
                            writer_flag = True
            for item1 in item.find_all(attrs={'class': 'columns small-9 large-10'}):
                for index1, ultag in enumerate(item1.find_all('ul', {'class': 'list-separator'})):
                    if (index1 == 0):
                        for index2, litag in enumerate(ultag.find_all('li')):
                            if (index2 == 2):
                                m_duration.append(litag.text.strip(' \n\t\r'))
                                duration_flag = True
                            if (index2 == 4):
                                for index3, litag2 in enumerate(litag.find_all('li')):
                                    if (index3 == 1):
                                        m_rating.append(litag2.text.strip(' \n\t\r'))
                                        rating_flag = True
                    elif (index1 == 1):
                        for index2, litag in enumerate(ultag.find_all('li')):
                            if (index2 == 0):
                                for index2, litag in enumerate(ultag.find_all(string=re.compile("Release Date:"))):
                                    if (release_date_flag == False):
                                        m_release_date.append(litag.parent.text.split("\n")[2].strip(" \n\t\r"))
                                        release_date_flag = True
        for index, item in enumerate(soup.find_all('p', attrs={'class': None})):
            if (index == 1):
                if (item != None):
                    m_storyline.append(item.text.strip(' \n\t\r').replace('...Read more', ''))
            else:
                m_storyline.append(None)
                storyline_flag = True
        for child in soup.find_all(string=re.compile("Genre:")):
            genre = []
            if(child.parent.text == "Genre:"):
                for index2, child2 in enumerate(child.parent.parent.find_all('li')):
                    if (index2 != 0):
                        genre.append(child2.text.strip(' \n\t\r'))
                m_genres.append(genre)
                genres_flag = True
        for child in soup.find_all(string=re.compile("Box Office:")):
            m_revenue.append(int(str(child.parent.find_next_sibling('li').text).replace("(more)", "").strip(' \n\t\r').replace(",", "")))
            revenue_flag = True
        if (duration_flag == False):
            m_duration.append(None)
        if (rating_flag == False):
            m_rating.append(None)
        if (release_date_flag == False):
            m_release_date.append(None)
        if (storyline_flag == False):
            m_storyline.append(None)
        if (revenue_flag == False):
            m_revenue.append(None)
        if (genres_flag == False):
            m_genres.append(None)
        if (director_flag == False):
            m_director_ID.append(None)
        if (writer_flag == False):
            m_writer_ID.append(None)

In [12]:
def retrieve_one_movie_actors_content(m_ids):
    actor_url = "https://elcinema.com/en/work/" + str(m_ids) + "/cast"
    return requests.get(actor_url).content

In [13]:
def retrieve_all_movies_actors_contents():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(retrieve_one_movie_actors_content, m_ids): m_ids for m_ids in m_IDs}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                actors_contents.append(data)
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                pass

In [14]:
def retrieve_actors_ids_from_contents():
    for i in range(len(actors_contents)):
        soup = BeautifulSoup(actors_contents[i])
        flag = False
        actors_ids = []
        for item in soup.find_all(string=re.compile("Actor")):
            w = item.parent.parent.find_next_sibling('div')
            if (w is not None):
                for x in w.find_all(attrs={'class': 'description'}):
                    actors_ids.append(x.find_all("li")[0].a['href'][-8:-1])
                flag = True
        if (flag == False):
            actors_ids.append(None)
        movie_actor_a_id.append(actors_ids)

In [15]:
def retrieve_one_castm_content(curl):
    return requests.get(curl).content

In [16]:
def retrieve_all_castm_contents():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(retrieve_one_castm_content, curl): curl for curl in cast_urls}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                cast_contents.append(data)
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                pass

In [17]:
def retrieve_cast_attributes_from_contents():
    for i in range(len(cast_contents)):
        soup = BeautifulSoup(cast_contents[i], 'html5lib')
        link = soup.find_all(href=True)
        cm_IDs.append(int(link[1]['href'][-8:-1]))
        biography_flag, birthdate_flag, nationality_flag = False, False, False
        for item in soup.find_all(attrs={'class': 'panel jumbo'}):
            name = item.find('span', attrs={'class':'left'})
            if (name != None):
                name = name.text.strip(' \n\t\r').split(" ", 1)
                if (len(name) == 0):
                    fname = None
                elif (len(name) == 1):
                    fname = name[0]
                    lname = None
                if (len(name) > 1):
                    fname = name[0]
                    lname = name[1]
            cm_first_name.append(fname)
            cm_last_name.append(lname)
        for item in soup.find_all(attrs={'class': 'intro-box'}):
            for item3 in item.find_all(attrs={'class': 'columns medium-9 large-5'}):
                for item4 in item3.find_all(attrs={'class': 'no-margin'}):
                    if (item4 is not None):
                        cm_biography.append(item4.text)
                        biography_flag = True
            for item1 in item.find_all(attrs={'class': 'list-separator list-title'}):
                not_birthdate, not_nationality = False, False
                for index, item2 in enumerate(item1.find_all('li')):
                    if (index == 0):
                        if (str(item2.text) != "Date of Birth:"):
                            not_birthdate = True
                        if (str(item2.text) != "Nationality:"):
                            not_nationality = True
                    elif (index == 1):
                        if (not_birthdate == False):
                            cm_birthdate.append(item2.text.strip(' \n\t\r'))
                            birthdate_flag = True
                        elif (not_nationality == False):
                            cm_nationality.append(item2.text.strip(' \n\t\r'))
                            nationality_flag = True
        if (biography_flag == False):
            cm_biography.append(None)
        if (birthdate_flag == False):
            cm_birthdate.append(None)
        if (nationality_flag == False):
            cm_nationality.append(None)

In [18]:
def preprocessing():
    for i in range(len(m_release_date)):
        if (m_release_date[i] is not None and m_release_date[i] != "None"):
            temp = m_release_date[i].split(" ")
            if (len(temp) == 3):
                if (int(temp[0]) < 10):
                    temp[0] = "0" + temp[0]
                if (int(temp[2]) < 10):
                    temp[2] = "0" + temp[2]
                temp = temp[0] + temp[1] + temp[2]
            else:
                temp = temp[0]
            m_release_date[i] = temp.strip(' \n\t').replace(" ", "").replace("-","").replace("January", "01").replace("February", "02").replace("March", "03").replace("April", "04").replace("May", "05").replace("June", "06").replace("July", "07").replace("August", "08").replace("September", "09").replace("October", "10").replace("November", "11").replace("December", "12")
            try:
                m_release_date[i] = datetime.datetime.strptime(m_release_date[i], '%Y%m%d').date()
            except:
                m_release_date[i] = datetime.datetime.strptime(m_release_date[i], '%d%m%Y').date()
    for i in range(len(cm_birthdate)):
        if (cm_birthdate[i] is not None and cm_birthdate[i]  != "None"):
            temp = cm_birthdate[i].split(" ")
            if (len(temp) == 3):
                if (int(temp[0]) < 10):
                    temp[0] = "0" + temp[0]
                if (int(temp[2]) < 10):
                    temp[2] = "0" + temp[2]
                temp = temp[0] + temp[1] + temp[2]
            else:
                temp = temp[0]
            cm_birthdate[i] = temp.strip(' \n\t').replace(" ", "").replace("-","").replace("January", "01").replace("February", "02").replace("March", "03").replace("April", "04").replace("May", "05").replace("June", "06").replace("July", "07").replace("August", "08").replace("September", "09").replace("October", "10").replace("November", "11").replace("December", "12")
            try:
                cm_birthdate[i] = datetime.datetime.strptime(cm_birthdate[i], '%Y%m%d').date()
            except:
                cm_birthdate[i] = datetime.datetime.strptime(cm_birthdate[i], '%d%m%Y').date()
    for i in range(len(m_duration)):
        m_duration[i] = m_duration[i].split(" ")[0] if m_duration[i] is not None else None
        m_duration[i] = None if m_duration[i] is not None and m_duration[i].lower().islower() else m_duration[i]
        m_duration[i] = int(m_duration[i]) if m_duration[i] is not None else None

In [19]:
def truncate_tables():
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("Truncate TABLE movies_genres")
    cursor.execute("Truncate TABLE movies_actors")
    cursor.execute("Truncate TABLE movie")
    cursor.execute("Truncate TABLE cast_member")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    db_connection.commit()
    db_connection.close()

In [20]:
def delete_duplicates_from_movie_genres():
    for i in range(len(m_IDs)):
        if (m_genres[i] is not None):
            for dup in sorted(list_duplicates(m_genres[i])):
                m_genres[i] = [k for n, k in enumerate(m_genres[i]) if k not in m_genres[i][:n]]

In [21]:
def fill_cast_member():
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    sql = "INSERT INTO cast_member (cm_ID, cm_first_name, cm_last_name, cm_biography, cm_birthdate, cm_nationality) VALUES (%s, %s, %s, %s, %s, %s)"
    for i in range(len(cm_IDs)):
        val = (cm_IDs[i], cm_first_name[i], cm_last_name[i], cm_biography[i], cm_birthdate[i], cm_nationality[i])
        cursor.execute(sql, val)
    db_connection.commit()
    print("Filled")
    db_connection.close()

In [22]:
def fill_movie():
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    sql = "INSERT INTO movie (m_ID, m_name, m_duration, m_rating, m_release_date, m_storyline, m_revenue, m_director_ID, m_writer_ID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    for i in range(len(m_IDs)):
        val = (m_IDs[i], m_name[i], m_duration[i], m_rating[i], m_release_date[i], m_storyline[i], m_revenue[i], m_director_ID[i], m_writer_ID[i])
        cursor.execute(sql, val)
    db_connection.commit()
    print("Filled")
    db_connection.close()

In [23]:
def fill_movies_genres():
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    sql = "INSERT INTO movies_genres (m_ID, genre) VALUES (%s, %s)"
    for i in range(len(m_IDs)):
        if (m_genres[i] is not None):
            for j in range(len(m_genres[i])):
                val = (m_IDs[i], m_genres[i][j])
                cursor.execute(sql, val)
    db_connection.commit()
    print("Filled")
    db_connection.close()

In [24]:
def fill_movies_actors():
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    sql = "INSERT INTO movies_actors (m_ID, cm_ID) VALUES (%s, %s)"
    for i in range(len(m_IDs)):
        if (movie_actor_a_id[i][0] is not None):
            for j in range(len(movie_actor_a_id[i])):
                val = (m_IDs[i], movie_actor_a_id[i][j])
                cursor.execute(sql, val)
    db_connection.commit()
    print("Filled")
    db_connection.close()

In [25]:
def top10_movies_by_revenue():
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT m_name, m_duration, m_rating, m_release_date, m_revenue FROM movie ORDER BY m_revenue DESC LIMIT 10")
    m = cursor.fetchall()
    params = ['m_name', 'm_duration', 'm_rating', 'm_release_date', 'm_revenue']
    df = pd.DataFrame(m, columns=params)
    print(df)

In [27]:
def find_movie_by_id(ID):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT m_name, m_duration, m_rating, m_release_date, m_storyline, m_revenue, c1.cm_first_name, c1.cm_last_name, c2.cm_first_name, c2.cm_last_name FROM movie INNER JOIN cast_member c1 ON movie.m_director_ID = c1.cm_ID INNER JOIN cast_member c2 ON movie.m_writer_ID = c2.cm_ID WHERE movie.m_ID='%s'" % ID)
    m = cursor.fetchall()
    director = m[0][-2] + " " + m[0][-1] if m[0][-1] is not None else m[0][-2]
    writer = m[0][-4] + " " + m[0][-3] if m[0][-3] is not None else m[0][-4]
    l = m[0][:-4]
    l = list(l)
    l.append(director)
    l.append(writer)
    print(l)

In [28]:
def find_movie_by_name(name):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT m_name, m_duration, m_rating, m_release_date, m_storyline, m_revenue, c1.cm_first_name, c1.cm_last_name, c2.cm_first_name, c2.cm_last_name FROM movie INNER JOIN cast_member c1 ON movie.m_director_ID = c1.cm_ID INNER JOIN cast_member c2 ON movie.m_writer_ID = c2.cm_ID WHERE movie.m_name='%s'" % name)
    m = cursor.fetchall()
    director = m[0][-2] + " " + m[0][-1] if m[0][-1] is not None else m[0][-2]
    writer = m[0][-4] + " " + m[0][-3] if m[0][-3] is not None else m[0][-4]
    l = m[0][:-4]
    l = list(l)
    l.append(director)
    l.append(writer)
    l = tuple(l)
    print(l)

In [29]:
def find_cast_member_by_id(ID):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT * FROM cast_member WHERE cm_ID='%s'" % ID)
    m = cursor.fetchall()
    name = m[0][1] + " " + m[0][2] if m[0][2] is not None else m[0][1]
    l = []
    l.append(name)
    l.append(m[0][3])
    l.append(m[0][4])
    l.append(m[0][5])
    print(l)

In [30]:
def find_cast_member_by_name(name):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    first = name.split(" ")[0]
    try:
        last = name.split(" ")[1]
    except:
        last = None
    cursor = db_connection.cursor()
    cursor.execute("SELECT * FROM cast_member WHERE cm_first_name='%s'" % first + " AND cm_last_name='%s'" % last)
    m = cursor.fetchall()
    name = m[0][1] + " " + m[0][2] if m[0][2] is not None else m[0][2]
    l = []
    l.append(name)
    l.append(m[0][2])
    l.append(m[0][3])
    l.append(m[0][4])
    print(l)

In [31]:
def find_unique_genres():
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT DISTINCT genre FROM movies_genres")
    m = cursor.fetchall()
    print(m)

In [32]:
def find_movies_of_a_specific_genre(genre):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT m_name FROM movie INNER JOIN movies_genres ON movie.m_ID=movies_genres.m_ID WHERE movies_genres.genre='%s'" % genre)
    m = cursor.fetchall()
    print(m)

In [131]:
def find_movies_of_a_cast_member_by_id(ID):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT CM.cm_first_name, CM.cm_last_name, M.m_name, M.m_duration, M.m_release_date, M.m_storyline FROM movies_actors MA JOIN movie M ON MA.m_ID = M.m_ID JOIN cast_member CM ON MA.cm_ID = CM.cm_ID WHERE MA.cm_ID='%s'" % ID)
    m = cursor.fetchall()
    name = m[0][0] + " " + m[0][1] if m[0][1] is not None else m[0][0]
    t = []
    for i in range(len(m)):
        l = []
        l.append(name)
        l.append(m[i][2])
        l.append(m[i][3])
        l.append(m[i][4])
        l.append(m[i][5])
        t.append(l)
    t = tuple(t)
    print(t)

In [110]:
def find_movies_of_a_cast_member_by_name(name):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    first = name.split(" ")[0]
    try:
        last = name.split(" ")[1]
    except:
        last = None
    cursor = db_connection.cursor()
    cursor.execute("SELECT M.m_name, M.m_duration, M.m_release_date, M.m_storyline FROM movie M JOIN movies_actors MA ON M.m_ID = MA.m_ID JOIN cast_member CM ON CM.cm_ID=MA.cm_ID WHERE CM.cm_first_name='%s'" % first + " AND CM.cm_last_name='%s'" % last)
    m = cursor.fetchall()
    t = []
    for i in range(len(m)):
        l = []
        l.append(name)
        l.append(m[i][0])
        l.append(m[i][1])
        l.append(m[i][2])
        l.append(m[i][3])
        t.append(l)
    t = tuple(t)
    print(t)

In [128]:
def add_user(email_address, username, gender, birthdate):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    sql = "INSERT INTO user (email_address, username, gender, birthdate) VALUES (%s, %s, %s, %s)"
    val = (email_address, username, gender, birthdate)
    cursor.execute(sql, val)
    db_connection.commit()

In [134]:
def add_review_by_movie_id(user_email_address, ID, textual_review):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    sql = "INSERT INTO review (user_email_address, m_ID, textual_review) VALUES (%s, %s, %s)"
    val = (user_email_address, ID, textual_review)
    cursor.execute(sql, val)
    db_connection.commit()

In [140]:
def add_review_by_movie_name(user_email_address, name, textual_review):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT m_ID from movie where m_name='%s'" % name)
    m = cursor.fetchall()
    ID = m[0][0]
    cursor = db_connection.cursor()
    sql = "INSERT INTO review (user_email_address, m_ID, textual_review) VALUES (%s, %s, %s)"
    val = (user_email_address, ID, textual_review)
    cursor.execute(sql, val)
    db_connection.commit()

In [152]:
def check_review_by_movie_id(ID):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT m_name FROM movie WHERE m_ID='%s'" % ID)
    name = cursor.fetchall()
    cursor.execute("SELECT user_email_address, textual_review FROM review WHERE review.m_ID='%s'" % ID)
    m = cursor.fetchall()
    l = []
    for i in range(len(m)):
        l = []
        l.append(name[0][0])
        l.append(m[i][0])
        l.append(m[i][1])
        print(l)

In [160]:
def check_review_by_movie_name(name):
    try:
        db_connection = MySQLdb.connect("sql11.freemysqlhosting.net", "sql11410479", "cSMqvaXALm", "sql11410479", charset = "utf8")
    except:
        print("Can't connect to database")
        return 0
    print("Connected")
    cursor = db_connection.cursor()
    cursor.execute("SELECT m_ID from movie where m_name='%s'" % name)
    m = cursor.fetchall()
    cursor.execute("SELECT user_email_address, textual_review FROM review WHERE review.m_ID='%s'" % m[0][0])
    m = cursor.fetchall()
    for i in range(len(m)):
        l = []
        l.append(name)
        l.append(m[i][0])
        l.append(m[i][1])
        print(l)

In [35]:
pages_contents = []

In [36]:
movies_contents = []

In [37]:
actors_contents = []

In [38]:
cast_contents = []

In [39]:
movies_urls = []

In [40]:
cast_urls = []

In [41]:
m_IDs, m_name, m_duration, m_rating, m_release_date, m_storyline, m_revenue, m_genres, m_director_ID, m_writer_ID = [], [], [], [], [], [], [], [], [], []

In [42]:
movie_actor_a_id = []

In [43]:
cm_ids, cm_urls, cm_IDs, cm_first_name, cm_last_name, cm_biography, cm_birthdate, cm_nationality = [], [], [], [], [], [], [], []

In [44]:
movies_table, movies_genres_table, castMembers_table, movies_actors_table = [], [], [], []

In [45]:
retrieve_all_pages_contents()

In [46]:
retrieve_movies_urls_from_contents()

In [47]:
print(len(movies_urls), "movies")
print("Duplicates:")
for dup in sorted(list_duplicates(movies_urls)):
    print(dup)
movies_urls = [i for n, i in enumerate(movies_urls) if i not in movies_urls[:n]]
print(len(movies_urls), "movies after deleating duplicates.")
print("Duplicates:")
for dup in sorted(list_duplicates(movies_urls)):
    print(dup)

688 movies
Duplicates:
('https://elcinema.com/en/work/1010351/', [230, 274])
('https://elcinema.com/en/work/2009614/', [609, 630])
('https://elcinema.com/en/work/2018611/', [599, 615])
('https://elcinema.com/en/work/2018826/', [529, 554])
('https://elcinema.com/en/work/2018996/', [530, 552])
('https://elcinema.com/en/work/2019219/', [233, 247, 275])
('https://elcinema.com/en/work/2020191/', [684, 686])
('https://elcinema.com/en/work/2026156/', [225, 246])
('https://elcinema.com/en/work/2026544/', [442, 446])
('https://elcinema.com/en/work/2028770/', [616, 620, 637])
('https://elcinema.com/en/work/2029316/', [490, 493])
('https://elcinema.com/en/work/2030327/', [245, 273])
('https://elcinema.com/en/work/2033958/', [367, 372])
('https://elcinema.com/en/work/2034424/', [226, 268])
('https://elcinema.com/en/work/2036658/', [366, 371, 373])
('https://elcinema.com/en/work/2039389/', [437, 444])
('https://elcinema.com/en/work/2040324/', [282, 303, 310])
('https://elcinema.com/en/work/2041312/

In [48]:
retrieve_all_movies_contents()

In [49]:
retrieve_movies_attributes_from_contents()

In [51]:
retrieve_all_movies_actors_contents()

In [52]:
retrieve_actors_ids_from_contents()

In [53]:
cm_ids_temp = m_director_ID + m_writer_ID 
for a_id in movie_actor_a_id:
    if type(a_id) is list:
        for item in a_id:
            cm_ids_temp.append(item)
    else:
        cm_ids_temp.append(a_id)
cm_ids = [i for i in cm_ids_temp if i]
cm_ids = [i for n, i in enumerate(cm_ids) if i not in cm_ids[:n]]
cast_urls = ["https://elcinema.com/en/person/" + s for s in cm_ids]

In [54]:
print(len(cm_ids))

5640


In [55]:
retrieve_all_castm_contents()

In [56]:
retrieve_cast_attributes_from_contents()

In [57]:
debugging()

Including nulls too; just making sure sizes match!
No. of pages-> 100
No. of movies-> 658
No. of occurances of existence of actors in movies-> 658
No. of cast members-> 5640
No. of movie names-> 658
No. of movie durations-> 658
No. of movie ratings-> 658
No. of movie release_dates-> 658
No. of movie storylines-> 1643
No. of movie revenues-> 658
No. of movie genres-> 658
No. of movie directors-> 658
No. of movie writers-> 658
No. of cast members 5640
No. of cast first names 5640
No. of cast last names 5640
No. of cast biography 5640
No. of cast birthdate 5640
No. of cast nationality 5640


In [58]:
saved_pages_contents = pages_contents
saved_movies_contents = movies_contents
saved_actors_contents = actors_contents
saved_cast_contents = cast_contents
saved_movies_urls = movies_urls
saved_cast_urls = cast_urls
saved_m_name = m_name
saved_m_duration = m_duration
saved_m_rating = m_rating
saved_m_release_date = m_release_date
saved_m_storyline = m_storyline
saved_m_revenue = m_revenue
saved_m_genres = m_genres
saved_m_director_ID = m_director_ID
saved_m_writer_ID = m_writer_ID
saved_movie_actor_a_id = movie_actor_a_id
saved_cm_ids = cm_IDs
saved_cm_urls = cm_urls
saved_cm_first_name = cm_first_name
saved_cm_last_name = cm_last_name
saved_cm_biography = cm_biography
saved_cm_birthdate = cm_birthdate
saved_cm_nationality = cm_nationality

In [59]:
temp_pages_contents = saved_pages_contents
temp_movies_contents = saved_movies_contents
temp_actors_contents = saved_actors_contents
temp_cast_contents = saved_cast_contents
temp_movies_urls = saved_movies_urls
temp_cast_urls = saved_cast_urls
temp_m_name = saved_m_name
temp_m_duration = saved_m_duration
temp_m_rating = saved_m_rating
temp_m_release_date = saved_m_release_date
temp_m_storyline = saved_m_storyline
temp_m_revenue = saved_m_revenue
temp_m_genres = saved_m_genres
temp_m_director_ID = saved_m_director_ID
temp_m_writer_ID = saved_m_writer_ID
temp_movie_actor_a_id = saved_movie_actor_a_id
temp_cm_ids = saved_cm_ids
temp_cm_urls = saved_cm_urls
temp_cm_first_name = saved_cm_first_name
temp_cm_last_name = saved_cm_last_name
temp_cm_biography = saved_cm_biography
temp_cm_birthdate = saved_cm_birthdate
temp_cm_nationality = saved_cm_nationality

In [60]:
preprocessing()

In [61]:
truncate_tables()

Connected


In [62]:
fill_cast_member()

Connected
Filled


In [63]:
fill_movie()

Connected
Filled


In [64]:
fill_movies_actors()

Connected
Filled


In [65]:
delete_duplicates_from_movie_genres()

In [66]:
fill_movies_genres()

Connected
Filled


In [67]:
top10_movies_by_revenue()

Connected
                              m_name m_duration  m_rating m_release_date  \
0                 El Feel El Azraq 2        130         8     2019-07-25   
1  Welad Rizk 2: A'wdat Osoud Al-Ard        128        12     2019-08-08   
2                         Casablanca        104        12     2019-06-04   
3                          El-Mamarr        150         0     2019-06-04   
4                           El Badla         96         0     2018-08-20   
5             Nadi El-Regal El-Serri        105        12     2019-01-23   
6                       Harb Karmooz        116        12     2018-06-16   
7                         El-Khaleya        120        12     2017-08-31   
8                     Horob Edterary        103        12     2017-06-25   
9                           Al-Felos        118         0     2019-12-24   

   m_revenue  
0  103633291  
1  100753422  
2   79156808  
3   74495223  
4   67339111  
5   59719314  
6   57634545  
7   56004004  
8   55551644  
9  

In [68]:
find_movie_by_id("2047945")

Connected
['Aman Ya Sahabi', '79', 12, datetime.date(2017, 8, 31), None, 4604152, 'Sayed El Sobky', 'Hany Hamdy']


In [69]:
find_movie_by_name("18 Youm")

Connected
('18 Youm', '125', 18, datetime.date(2011, 5, 19), 'In the midst of the chaos of Egypt’s Arab Spring, filmmaker Nada Riyadh falls in love with Ayman. The initial euphoria makes way for doubt, anxiety and blame – a metaphor for the disappointing trajectory of events in Egypt. When Ayman decides to leave the country because of the deteriorating social and political conditions, she does everything she can to convince him to stay. The escalating tension puts her own commitment to remain in the country to the test.', None, 'Sherif Arafa', 'Sherif Arafa')


In [70]:
find_cast_member_by_id("1100178")

Connected
['Hasan Al-Raddad', 'Hassan is the son of renowned Egyptian actor Nabil el-Raddad and his full name is Hassan Nabil el-Raddad. The actor was born in the city of Port Said and his debut appearance came about as a minor role in the title “Kheyanna Mashrou’a” (“A Legitimate Betrayal”). Thereafter he...Read more assumed a role in the first part of the series “al-Daly” as Bassem al-Daly who was Sa’ad al-Daly’s middle child. He reappeared once again in the second part of that series. The actor made another appearance in the series “Ibn al-Arnadly” where he portrayed ‘Aly al-Arnadly, the son of Yahya al-Fakhran.', datetime.date(1984, 1, 2), 'Egypt']


In [71]:
find_cast_member_by_name("Ahmed Helmy")

Connected
['Ahmed Helmy', 'Helmy', 'Ahmed Helmy was born in Banha. He graduated from the Higher Institute of Theatrical Arts. He moved to Cairo to pursue a diverse career including interior design and media production. He later on settled on the media path, working as a host for multiple TV programs mainly aired on...Read more Egyptian satellite networks. Following his successful and popular TV run, he ventured into feature films. His first acting job was in "Aboud Ala Hodood" (Aboud on the Borders) (1999), which proved to be his huge breakout performance. Since then, Helmy has become a well-established comedian and a leading man. He\'s starred in "Omar 2000" (2000), "Sahar El Layali" (Sleepless Nights) (2003), and most recently "Assal Eswed" (Black Honey) (2010) and 2013\'s "Ala Gothety" (Over My Dead Body). Ahmed Helmy is married to actress Mona Zaky and they have one daughter, Leila.', datetime.date(1969, 11, 18)]


In [72]:
find_movies_of_a_specific_genre("Comedy")

Connected
(('Touti Fruti',), ('El- Haram Street',), ('Tita Rahiba',), ('365 Days of Happiness',), ('Please Call Me',), ('Bibo and Beshir',), ('Fasel Wa Na‘oud',), ('Muhtaram Iila Rubue',), ('Me Or Him',), ("Banat ElA'm",), ('EUC',), ('X Large',), ('Al-Rajul Al-Ghamid Bisalamatoh',), ('El Haramy wel Abeet',), ('Assal Eswed',), ("Wahed Sa'idy",), ('Lembi 8 Giga',), ('Ibn Alqonsol',), ('Geddo Habiby',), ('Omar wa Salma 3',), ('Cry of An Ant',), ('Ya Tahadi Ya Taadi',), ('Yabani Asli',), ('The Elephant in a Handkerchief: Said Harakat',), ("Eza'et Hobb",), ('La Taragoa Wala Esteslam (El Kabda El Dameyya)',), ('Bonne Soirée',), ('Walad albalad',), ('El Talata Yeshtghalonha',), ('Bolbol Hayran',), ('Sameer, Shaheer & Baheer',), ('Ali Baba',), ('Samy Carbon Oxide',), ("Mickey's Family",), ('Ana Badea Ya Wadia',), ("Alzheimer's",), ('Hello...Cairo',), ('18 Youm',), ('Fokak Menny',), ('Tek Tek Boom',), ('Game Over',), ('Ghesh El Zawgeya',), ('Dawlat Security',), ('Cima Ali Baba',), ('Helm Aziz',

In [132]:
find_movies_of_a_cast_member_by_id("1100178")

Connected
(['Hasan Al-Raddad', 'Ana Badea Ya Wadia', '90', datetime.date(2011, 8, 30), None], ['Hasan Al-Raddad', 'My Brother the Devil', '111', datetime.date(2012, 11, 9), None], ['Hasan Al-Raddad', 'Al-Anissa Mamy', '100', datetime.date(2012, 10, 24), None], ['Hasan Al-Raddad', 'Nazareyet Amity', '109', datetime.date(2013, 8, 7), None], ['Hasan Al-Raddad', 'Antar & Beesa', '90', datetime.date(2014, 7, 28), None], ['Hasan Al-Raddad', 'أهل العيب', None, None, None], ['Hasan Al-Raddad', 'Noum Eltalat', '96', datetime.date(2015, 7, 16), None], ['Hasan Al-Raddad', 'Akhdar Yabis', '73', datetime.date(2017, 10, 25), None], ['Hasan Al-Raddad', "Al-Khawaga's Dilemma", '110', datetime.date(2018, 1, 6), 'A father who had big dreams for his three children, returns after many years, only to find them living a different life from what he had envisioned. As the three brothers get involved in a terrorist operation, they try to stop it with the help of a group of girls.'], ['Hasan Al-Raddad', 'El Bad

In [112]:
find_movies_of_a_cast_member_by_name("Ahmed Helmy")

Connected
(['Ahmed Helmy', 'X Large', '130', datetime.date(2011, 11, 2), None], ['Ahmed Helmy', 'Assal Eswed', '130', datetime.date(2010, 5, 25), None], ['Ahmed Helmy', 'Bolbol Hayran', '82', datetime.date(2010, 11, 11), 'The film tells the story of a female police officer who faces sexual harassment and decides to fight this widespread phenomenon. She is assigned to arrest sexual harassers in public places.'], ['Ahmed Helmy', '18 Youm', '125', datetime.date(2011, 5, 19), 'In the midst of the chaos of Egypt’s Arab Spring, filmmaker Nada Riyadh falls in love with Ayman. The initial euphoria makes way for doubt, anxiety and blame – a metaphor for the disappointing trajectory of events in Egypt. When Ayman decides to leave the country because of the deteriorating social and political conditions, she does everything she can to convince him to stay. The escalating tension puts her own commitment to remain in the country to the test.'], ['Ahmed Helmy', 'روليت', None, None, None], ['Ahmed Hel

In [130]:
add_user("mai@hotmail.com", "mai_11", "F", datetime.datetime(2000, 2, 12).date())

Connected


In [136]:
add_review_by_movie_id("mai@hotmail.com", "1644198", "Liked el sakka")

Connected


In [141]:
add_review_by_movie_name("mai@hotmail.com", "X Large", "great from Helmy")

Connected


In [153]:
check_review_by_movie_id("1644198")

Connected
['Al-Maslaha', 'mai@hotmail.com', 'Liked el sakka']


In [161]:
check_review_by_movie_name("X Large")

Connected
['X Large', 'mai@hotmail.com', 'great from Helmy']
